In [ ]:
# Importing all the required libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from matplotlib import pyplot as plt
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Loading the all the data set
train_identity = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_identity.csv')
train_trans = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')

test_identity = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_identity.csv')
test_trans = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_transaction.csv')

### Basic Data Summary


In [ ]:
# Data Shape
print('Data Shape of train identity :', train_identity.shape)
print('Data Shape of train transaction :', train_trans.shape)

print('Data Shape of test identity :', test_identity.shape)
print('Data Shape of test transaction :', test_trans.shape)

In [ ]:
# Data Overview
print("Train Identity")
train_identity.head(10).T

In [ ]:
# Data Overview
print("Train Transaction")
train_trans.head(10).T

In [ ]:
# Merging the dataset
train_merged = pd.merge(train_trans, train_identity, on = 'TransactionID')
test_merged = pd.merge(test_trans, test_identity, on = 'TransactionID')

In [ ]:
del train_identity, train_trans, test_identity, test_trans

In [ ]:
# Target in Analysis
plt.subplots(figsize = (6,6))
train_merged['isFraud'].value_counts().plot('pie')
plt.show()

Target : isFraud is highly imbalanced which is common in real world problems

In [ ]:
# Missing Values
missing_values = ((train_merged.isnull().sum()/len(train_merged)).sort_values(ascending = False))*100
print("Missing Values")
print(missing_values)

In [ ]:
# Removing all the variables having more than 90% of the missing value
cols_to_remove = missing_values[missing_values>=90].index.tolist()

In [ ]:
# Removing above columns from train and test data
train_merged.drop(cols_to_remove, axis =1, inplace = True)
test_merged.drop(cols_to_remove, axis =1, inplace = True)

In [ ]:
train_merged.head(10).T

In [ ]:
# Splitting data into categorical and numerical
categorial_data = train_merged.select_dtypes(include = ['object'])
numerical_data = train_merged.select_dtypes(exclude = ['object'])

In [ ]:
categorial_data.head(10).T

# Analysis of Categorical Data


In [ ]:
categorial_data.describe().T

In [ ]:
# ProductCD Analysis
fig, ax = plt.subplots(1, 2, figsize=(14, 6))
sns.countplot('ProductCD', data = categorial_data, ax=ax[0])
sns.countplot(categorial_data['ProductCD'], hue = numerical_data['isFraud'], ax=ax[1])
plt.subplots_adjust(wspace = 0.5)
plt.show()

ProductCD has only 4 categorical levels , will do one hot encoding of ProductCD


In [ ]:
# Card4 Analysis
fig, ax = plt.subplots(1, 2, figsize=(14, 6))
sns.countplot('card4', data = categorial_data, ax=ax[0])
sns.countplot(categorial_data['card4'], hue = numerical_data['isFraud'], ax=ax[1])
plt.subplots_adjust(wspace = 0.5)
plt.show()

In [ ]:
# Card6 Analysis
fig, ax = plt.subplots(1, 2, figsize=(14, 6))
sns.countplot('card6', data = categorial_data, ax=ax[0])
sns.countplot(categorial_data['card6'], hue = numerical_data['isFraud'], ax=ax[1])
plt.subplots_adjust(wspace = 0.5)
plt.show()

In [ ]:
# P_emaildomain
plt.figure(figsize=(20, 6))
sns.countplot('P_emaildomain', data = categorial_data)
plt.xticks(rotation=90)
plt.show()

1. Top 3 P_emaildomain used are
1. gmail.com
2. hotmail.com
3. anonymous.com

> We can combine few P_emaildomain into one based on their frequency and we can combine gmail and gmail.com

In [ ]:
temp = pd.DataFrame(categorial_data['P_emaildomain'].value_counts())
temp['%cent'] = temp['P_emaildomain']/temp['P_emaildomain'].sum()*100
temp['others'] = np.where(temp['%cent'] > 1, temp.index, 'others')
p_email_mapping = temp['others'].to_dict()
categorial_data['P_emaildomain_new'] = categorial_data['P_emaildomain'].map(p_email_mapping)

In [ ]:
# P_email and P_email_new Analysis
fig, ax = plt.subplots(1, 2, figsize=(20, 6))
categorial_data['P_emaildomain'].value_counts().plot(kind = 'bar', ax = ax[0])
categorial_data['P_emaildomain_new'].value_counts().plot(kind = 'bar', ax = ax[1])
plt.show()

In [ ]:
# R_emaildomain
plt.figure(figsize=(20, 6))
categorial_data['R_emaildomain'].value_counts().plot(kind = 'bar')
plt.show()

In [ ]:
# M4
# R_emaildomain
plt.figure(figsize=(20, 6))
categorial_data['M4'].value_counts().plot(kind = 'pie')
plt.show()

M4 has only three categories in it with M2 level has in majority.

In [ ]:
# Taking all the variable started with id
id_cols = categorial_data.columns[categorial_data.columns.str.startswith('id')].tolist()
# Removing id_30, id_31, id_33: Want to look at them separately due to high number of levels in it
cols_to_remove = ['id_30', 'id_31', 'id_33']
for i in cols_to_remove:
    id_cols.remove(i)

In [ ]:
fig, ax = plt.subplots(5, 2, figsize = (15, 20),
                       gridspec_kw={'hspace' : 0.5, 'wspace':0.2})
ax = np.reshape(ax, (10))

for col, axis in zip(id_cols, ax):
    sns.countplot(categorial_data[col], ax = axis)

Most of the data in columns starting with "id" is binary and having values like
1. Found and Not Found
2. T and F 

In [ ]:
#cols_to_remove = ['id_30', 'id_31', 'id_33']
# id_30
plt.figure(figsize=(20, 6))
categorial_data['id_30'].value_counts().plot(kind = 'bar')
plt.show()

The most occuring level is windows. I will create one more variable where I will keep only Windows, MAC OS, iOS

# WIP 